# Tracing and Datasets with LangChainPlus

LangChain makes it easy to get started with Agents and other LLM applications. However, it can be tricky to get right, especially when you need to deliver a full product. To speed up your application development process, and to help monitor your applications in production, LangChain offers additional tracing and tooling.

When might you want to use tracing? Some situations we've found it useful include:
- Quickly debugging a new chain, agent, or set of tools
- Evaluating a given chain across different LLMs or Chat Models to compare results or improve prompts
- Running a given chain multiple time on a dataset to ensure it consistently meets a quality bar.


In this notebook, we'll show how to enable tracing in your LangChain applications and walk you a couple common ways to evaluate your agents.
We'll focus on using Datasets to benchmark Chain behavior.

**Bear in mind that this notebook is designed under the assumption that you're running the latest LangChain+ server locally in the background. This is done using the folowing command in your terminal:**


```
pip install --upgrade langchain
langchain plus start
```

We also have a hosted version which is in private beta. We will share more details as it progresses.

Now, let's get started by creating a client to connect to LangChain+.

## Setting up Tracing

The V2 tracing API can be activated by setting the `LANGCHAIN_TRACING_V2` environment variable to true. Assuming you've successfully initiated the server as described earlier, running LangChain Agents, Chains, LLMs, and other primitives will automatically start capturing traces. Let's begin our exploration with a straightforward math example.

**NOTE**: You must also set your `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables in order to run the following tutorial.


**NOTE:** You can also use the `tracing_v2_enabled` context manager to capture sessions within a given context:
```
from langchain.callbacks.manager import tracing_v2_enabled
with tracing_v2_enabled("My Session Name"):
    ...
```

**NOTE:** You can optionally set the `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables if using the hosted version which is in private beta.

In [1]:
import os
from langchainplus_sdk import LangChainPlusClient

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_SESSION"] = "Tracing Walkthrough"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"  # Uncomment this line if you want to use the hosted version
# os.environ["LANGCHAIN_API_KEY"] = "<YOUR-LANGCHAINPLUS-API-KEY>"  # Uncomment this line if you want to use the hosted version.

client = LangChainPlusClient()
print("You can click the link below to view the UI")
client

You can click the link below to view the UI


LangChainPlusClient (API URL: https://dev.api.langchain.plus)

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [3]:
import asyncio

inputs = [
    "How many people live in canada as of 2023?",
    "who is dua lipa's boyfriend? what is his age raised to the .43 power?",
    "what is dua lipa's boyfriend age raised to the .43 power?",
    "how far is it from paris to boston in miles",
    "what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?",
    "what was the total number of points scored in the 2023 super bowl raised to the .23 power?",
    "how many more points were scored in the 2023 super bowl than in the 2022 super bowl?",
    "what is 153 raised to .1312 power?",
    "who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?",
    "what is 1213 divided by 4345?",
]
results = []


async def arun(agent, input_example):
    try:
        return await agent.arun(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


for input_example in inputs:
    results.append(arun(agent, input_example))
results = await asyncio.gather(*results)

## Creating the Dataset

Now that you've captured a session entitled 'Tracing Walkthrough', it's time to create a dataset. We will do so using the `create_dataset` method below.

In [4]:
dataset_name = "calculator-example-dataset"

In [5]:
if dataset_name in set([dataset.name for dataset in client.list_datasets()]):
    client.delete_dataset(dataset_name=dataset_name)
dataset = client.create_dataset(
    dataset_name, description="A calculator example dataset"
)
runs = client.list_runs(
    session_name=os.environ["LANGCHAIN_SESSION"],
    execution_order=1,  # Only return the top-level runs
    error=False,  # Only runs that succeed
)
for run in runs:
    try:
        client.create_example(
            inputs=run.inputs, outputs=run.outputs, dataset_id=dataset.id
        )
    except:
        pass

**Alternative: Creating a Dataset in the UI** 

Alternatively, you could create or edit the dataset in the UI using the following steps:

   1. Navigate to the UI by clicking on the link below.
   2. Select the 'search_and_math_chain' session from the list.
   3. Next to the fist example, click "+ to Dataset".
   4. Click "Create Dataset" and create a title **"calculator-example-dataset"**.
   5. Add the other examples to the dataset as well

Once you've used LangChain+ for a while, you will have a number of datasets to work with. To view all saved datasets, execute the following code:

```
datasets = client.list_datasets()
print(datasets)
```


**Optional:** If you didn't run the trace above, you can also create datasets by uploading dataframes or CSV files.

In [6]:
# !pip install datasets > /dev/null
# !pip install pandas > /dev/null

In [7]:
# import pandas as pd
# from langchain.evaluation.loading import load_dataset

# dataset = load_dataset("agent-search-calculator")
# df = pd.DataFrame(dataset, columns=["question", "answer"])
# df.columns = ["input", "output"] # The chain we want to evaluate below expects inputs with the "input" key
# df.head()

In [8]:
# dataset_name = "calculator-example-dataset"

# if dataset_name not in set([dataset.name for dataset in client.list_datasets()]):
#     dataset = client.upload_dataframe(df,
#                             name=dataset_name,
#                             description="A calculator example dataset",
#                             input_keys=["input"],
#                             output_keys=["output"],
#                    )

## Running a Chain on a Traced Dataset

Once you have a dataset, you can run a compatible chain or other object over it to see its results. The run traces will automatically be associated with the dataset for easy attribution and analysis.

**First, we'll define the chain we wish to run over the dataset.**

In this case, we're using an agent, but it can be any simple chain.

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

**Now we're ready to run the chain!**

The docstring below hints ways you can configure the method to run.

In [10]:
from langchain.client import arun_on_dataset
?arun_on_dataset

Signature:
arun_on_dataset(
    dataset_name: 'str',
    llm_or_chain_factory: 'MODEL_OR_CHAIN_FACTORY',
    *,
    concurrency_level: 'int' = 5,
    num_repetitions: 'int' = 1,
    session_name: 'Optional[str]' = None,
    verbose: 'bool' = False,
    client: 'Optional[LangChainPlusClient]' = None,
) -> 'Dict[str, Any]'
Docstring:
Run the chain on a dataset and store traces to the specified session name.

Args:
    client: Client to use to read the dataset.
    dataset_name: Name of the dataset to run the chain on.
    llm_or_chain_factory: Language model or Chain constructor to run
        over the dataset. The Chain constructor is used to permit
        independent calls on each example without carrying over state.
    concurrency_level: The number of async tasks to run concurrently.
    num_repetitions: Number of times to run the model on each example.
        This is useful when testing success rates or generating confidence
        intervals.
    session_name: Name of the session

In [11]:
# Since chains can be stateful (e.g. they can have memory), we need provide
# a way to initialize a new chain for each row in the dataset. This is done
# by passing in a factory function that returns a new chain for each row.
chain_factory = lambda: initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
)

# If your chain is NOT stateful, your lambda can return the object directly
# to improve runtime performance. For example:
# chain_factory = lambda: agent

In [12]:
evaluation_session_name = "Search + Calculator Agent Evaluation"
chain_results = await arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain_factory,
    concurrency_level=5,  # Optional, sets the number of examples to run at a time
    verbose=True,
    session_name=evaluation_session_name,  # Optional, a unique session name will be generated if not provided
    client=client,
)

# Sometimes, the agent will error due to parsing issues, incompatible tool inputs, etc.
# These are logged as warnings here and captured as errors in the tracing UI.

Chain failed for example 59fb1b4d-d935-4e43-b2a7-bc33fde841bb. Error: LLMMathChain._evaluate("
round(0.2791714614499425, 2)
") raised error: 'VariableNode' object is not callable. Please try again with a valid numerical expression


### Reviewing the Chain Results

You can review the results of the run in the tracing UI below and navigating to the session 
with the title **"Search + Calculator Agent Evaluation"**

In [13]:
# You can navigate to the UI by clicking on the link below
client

LangChainPlusClient (API URL: https://dev.api.langchain.plus)

## Running an Evaluation Chain

Manually comparing the results of chains in the UI is effective, but it can be time consuming.
It's easier to leverage AI-assisted feedback to evaluate your agent's performance.

A few ways of doing this include:
- Adding ground-truth answers as outputs to the dataset and evaluating relative to those references.
- Evaluating the overall agent trajectory based on the tool usage and intermediate steps.
- Evaluating performance based on 'context' such as retrieved documents or tool results.
- Evaluating 'aspects' of the agent's response in a reference-free manner using targeted agent prompts.
    
Below, we show how to run an evaluation chain that compares the model output with the ground-truth answers.

**Note: the feedback API is currently experimental and subject to change.**

In [14]:
from langchain.evaluation.run_evaluators import get_qa_evaluator, get_criteria_evaluator
from langchain.chat_models import ChatOpenAI

eval_llm = ChatOpenAI(temperature=0)

qa_evaluator = get_qa_evaluator(eval_llm)
helpfulness_evaluator = get_criteria_evaluator(eval_llm, "helpfulness")
conciseness_evaluator = get_criteria_evaluator(eval_llm, "conciseness")
custom_criteria_evaluator = get_criteria_evaluator(eval_llm, {"fifth-grader-score": "Do you have to be smarter than a fifth grader to answer this question?"})

evaluators = [qa_evaluator, helpfulness_evaluator, conciseness_evaluator, custom_criteria_evaluator]

In [27]:
from tqdm.notebook import tqdm
feedbacks = []
runs = client.list_runs(session_name=evaluation_session_name, execution_order=1, error=False)
for run in tqdm(runs):
    eval_feedback = []
    for evaluator in evaluators:
        eval_feedback.append(client.aevaluate_run(run, evaluator))
    feedbacks.extend(await asyncio.gather(*eval_feedback)) 

0it [00:00, ?it/s]

In [29]:
client

LangChainPlusClient (API URL: https://dev.api.langchain.plus)